In [2]:
# function to convert points from cartesian Part Coordiante System to cartesian Machine Coordinate System 
# input:  mcs: 4x4 transformation matrix
#         pts: nx3 list of points in PCS
# output: nx3 list of points in MCS
function pcs_to_mcs_xyz(mcs, pts)
    # transform all points into LR Cartesian coordinates
    # homogenize the pts with a 1
    qty = size(pts,1)
    tmp = [pts ones(qty,1)]
    # multiply by inverse of the LR's location/orientation
    ret = inv(mcs) * tmp'
    return ret[1:3,:]'
end;

# function to convert points from cartesian Part Coordiante System to cartesian Machine Coordinate System 
# input:  mcs: 4x4 transformation matrix
#         pts: nx3 list of points in MCS
# output: nx3 list of points in PCS
function mcs_to_pcs_xyz(mcs, pts)
    # transform all points into LR Cartesian coordinates
    # homogenize the pts with a 1
    qty = size(pts,1)
    tmp = [pts ones(qty,1)]
    # multiply by inverse of the LR's location/orientation
    ret = mcs * tmp'
    return ret[1:3,:]'
end;

# function to convert points from cartesian to spherical
# input:  pts: nx3 list of xyz (cartesian) points
# output: nx3 list of spherical points (range, theta, phi)
function xyz_to_sph(pts)
    r = [vecnorm(pts[i,:]) for i=1:size(pts,1)]
    r2= [vecnorm(pts[i,1:2]) for i=1:size(pts,1)]
    t = atan2.(r2, pts[:,3])
    p = atan2.(pts[:,2], pts[:,1])
    return [r t p]
end;

# function to convert points from spherical to cartesian
# input:  pts: nx3 list of (r,t,p) (spherical) points
# output: nx3 list of cartesian points (xyz)
function sph_to_xyz(pts)
    # multiply by inverse of the LR's location/orientation
    x = pts[:,1] .* sin.(pts[:,2]) .* cos.(pts[:,3]) # rsinθcosϕ
    y = pts[:,1] .* sin.(pts[:,2]) .* sin.(pts[:,3]) # rsinθsinϕ
    z = pts[:,1] .* cos.(pts[:,2])                   # rcosθ
    return [x y z]
end;

# load the feature file
data = readcsv("50_Holes_FrontLeft.csv");
pts_pcs = convert(Array{Float64, 2}, data[:,2:4]) # feature locations in PCS
lr_tfm = readcsv("Location1.csv") # transformation matrix for LR's position

# convert to spherical
pts_mcs = pcs_to_mcs_xyz(lr_tfm, pts_pcs) # convert feature locations to MCS
pts_lr = xyz_to_sph(pts_mcs) # convert feature locations to spherical

# round trip as a test
pts_mcs_back = sph_to_xyz(pts_lr)
pts_pcs_back = mcs_to_pcs_xyz(lr_tfm, pts_mcs_back)

# points should be the same
display([pts_pcs pts_pcs_back])


51×6 Array{Float64,2}:
  0.576372  1.28479    2.18214    0.576372  1.28479    2.18214 
  0.363344  0.944167   1.24128    0.363344  0.944167   1.24128 
 -0.820859  0.787889   0.985161  -0.820859  0.787889   0.985161
  0.535124  1.28018    2.25283    0.535124  1.28018    2.25283 
  0.576315  1.28427    2.18212    0.576315  1.28427    2.18211 
  0.599436  1.28617    2.15049    0.599436  1.28617    2.15049 
  0.626194  1.28575    2.13824    0.626194  1.28575    2.13824 
  0.652517  1.29513    2.08841    0.652517  1.29513    2.08841 
  0.767131  1.30497    1.96929    0.767131  1.30497    1.96929 
  0.766731  1.32892    1.80089    0.766731  1.32892    1.80089 
  0.766751  1.33604    1.65115    0.766751  1.33604    1.65115 
  0.289392  1.39443    1.07905    0.289392  1.39443    1.07905 
  0.110988  1.39614    1.08797    0.110988  1.39614    1.08797 
  ⋮                                                    ⋮       
  0.508046  0.969538   2.262      0.508046  0.969538   2.262   
  0.782297  0.949